
## Examples
https://github.com/awslabs/amazon-bedrock-agent-samples/blob/main/examples/amazon-bedrock-agents/human_resources_agent/create-agent-with-function-definition-roc-and-user-confirmation/create-agent-with-function-definition-roc-and-user-confirmation.ipynb
https://docs.aws.amazon.com/bedrock/latest/userguide/agents-api-schema.html




In [1]:
import boto3
import pprint, json, time, uuid

In [2]:
# Clients
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

session = boto3.session.Session()


NoRegionError: You must specify a region.

In [ ]:
# Constants
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
pprint.print(region)
pprint.print(account_id)

agent_foundation_model = "tbd-bra-model-name"
agent_bedrock_allow_policy_name = "tbd-bra-iam-policy-name"
agent_role_name = "tbd-bra-iam-role-name"

In [ ]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

In [ ]:
# Create agent 
agent_name = "tbd-bra-name"
agent_description = "tbd-bra-description"
agent_instruction = "tbd-bra-instructions"


response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response


In [ ]:
agent_id = response['agent']['agentId']
agent_id

# Function schema vs OpenAPI Schema 


In [ ]:
import sqlite3
from datetime import datetime
    
def reserve_vacation_time(employee_id, start_date, end_date):
    try:
        # Fetch from Databricks SQL Warehouse based UC function execution 
        return "string"
    except Exception as e:
        raise Exception(f"Error occurred: {e}")

In [ ]:
# Function schema vs OpenAPI Schema 

agent_functions = [

    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        },
        'requireConfirmation':'ENABLED'
    },
]

In [5]:
# OpenAPI Schema YAML file content 
openapi_schema_string = """
openapi: 3.0.0
info:
  title: GetWeather API
  version: 1.0.0
  description: gets weather
paths:
  /getWeather/{location}/:
    get:
      summary: gets weather in Celsius
      description: gets weather in Celsius
      operationId: getWeather
      parameters:
        - name: location
          in: path
          description: location name
          required: true
          schema:
            type: string
      responses:
        "200":
          description: weather in Celsius
          content:
            application/json:
              schema:
                type: string
"""

'\nopenapi: 3.0.0\ninfo:\n  title: GetWeather API\n  version: 1.0.0\n  description: gets weather\npaths:\n  /getWeather/{location}/:\n    get:\n      summary: gets weather in Celsius\n      description: gets weather in Celsius\n      operationId: getWeather\n      parameters:\n        - name: location\n          in: path\n          description: location name\n          required: true\n          schema:\n            type: string\n      responses:\n        "200":\n          description: weather in Celsius\n          content:\n            application/json:\n              schema:\n                type: string\n'

In [ ]:
# Prepare agent group using functionSchema
agent_action_group_name = "tbd-bda-action-group-name"
agent_action_group_description = "tbd-bda-action_group_description"

agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
# Prepare agent group using OpenAPI Schema
# API schema as a S3 file
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    actionGroupName=agent_action_group_name,
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description=agent_action_group_description
)

In [ ]:
# API schema as payload
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    actionGroupName=agent_action_group_name,
    apiSchema={
        'payload': openapi_schema_string
    },
    description=agent_action_group_description
)

In [ ]:
# Invoke agent 
# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False
# Pause to make sure agent alias is ready
# time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Book 2 days off for employee 1 with the start date of 2024-11-14 and the end date of 2024-11-19",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

event_stream = agentResponse['completion']

for event in event_stream:
    if 'returnControl' in event:
        pprint.pp(event)
        print(event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["function"])

In [ ]:
reserve_vacation_time_msg = None
for invocationInput in event["returnControl"]["invocationInputs"]:
    function_to_call = invocationInput["functionInvocationInput"]["function"]
    if function_to_call == "reserve_vacation_time":
        employee_id = None
        start_date = None
        end_date = None
        for param in invocationInput["functionInvocationInput"]["parameters"]:
            if param["name"] == "employee_id":
                employee_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]
                
        if employee_id and start_date and end_date:
            reserve_vacation_time_msg = reserve_vacation_time(employee_id, start_date, end_date)

reserve_vacation_time_msg

In [ ]:
# Invoke agent with function responses 

raw_response_with_roc = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    sessionState={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
                'functionResult': {
                    'actionGroup': event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["actionGroup"],
                    'function': event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["function"],
                    'confirmationState': 'CONFIRM',
                    'responseBody': {
                        "TEXT": {
                            'body': "reserve_vacation_time: "+str(reserve_vacation_time_msg)
                        }
                    }
                }
        }]}
)

print(raw_response_with_roc)